In [20]:
import os
import pandas as pd
from datetime import datetime

In [16]:
siglas = pd.read_csv('Utils/estados_siglas.csv', usecols=['Sigla', 'Estado'])
uf_pes_cecad = pd.read_csv('Data/Pessoas/UF_CECAD.csv')
uf_fam_cecad = pd.read_csv('Data/Fam/UF_CECAD.csv')
br_fam_cecad = pd.read_csv('Data/Fam/BR_CECAD.csv')
br_pes_cecad = pd.read_csv('Data/Pessoas/BR_CECAD.csv')

In [4]:
def to_date(df):
    df['data'] = df['ano'].astype(str) + '-' + df['mes'].apply(lambda x: '0' + str(x) if x < 10 else str(x))
    df['data'] = pd.to_datetime(df['data'], format='%Y-%m')

In [5]:
dir = os.listdir("Data/Fam/")
total = len(dir)
dfs=[]
for file in dir:
    df = pd.read_csv(f"Data/Fam/{file}", header=0, index_col='Código')
    df['ano'] = df['Referência'].apply(lambda x: int(x.split('/')[1]))
    df['mes'] = df['Referência'].apply(lambda x: int(x.split('/')[0]))
    df.drop(columns=['Referência'], inplace=True)
    df.set_index(['UF','Unidade Territorial','ano','mes'], inplace=True)
    dfs.append(df)
    print('Arquivo lido. Restam', total-len(dfs), 'arquivos...')
df = pd.concat(dfs, axis=1)


Arquivo lido. Restam 5 arquivos...
Arquivo lido. Restam 4 arquivos...
Arquivo lido. Restam 3 arquivos...
Arquivo lido. Restam 2 arquivos...
Arquivo lido. Restam 1 arquivos...
Arquivo lido. Restam 0 arquivos...


In [6]:
df.rename(columns={
    'Famílias não beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de extrema pobreza':'fam_nao_benef_ext_pobreza',
    'Famílias beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de pobreza':'fam_benef_pobreza',
    'Famílias beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de extrema pobreza':'fam_benef_ext_pobreza',
    'Famílias não beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de pobreza':'fam_nao_benef_pobreza',
    'Famílias inscritas no Cadastro Único em situação de extrema pobreza':'fam_inscritas_ext_pobreza',
    'Famílias inscritas no Cadastro Único em situação de pobreza':'fam_insc_pobreza',
    },
    inplace = True)

In [7]:
df['demanda_reprimida_total'] = df['fam_nao_benef_pobreza'] + df['fam_nao_benef_ext_pobreza']

In [8]:
uf = df.groupby(['ano','mes','UF']).sum()

In [9]:
br = df.groupby(['ano','mes']).sum()

In [10]:
df.reset_index(inplace=True)
uf.reset_index(inplace=True)
br.reset_index(inplace=True)

In [33]:
uf.head()

,ano,mes,UF,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total,data,nao_recebe_pessoas,recebe_pessoas,sem_resposta,total_pessoas,Estado
0,2012,8,AC,0.0,0.0,0.0,0.0,66434,24358,0.0,2012-08-01,NaN,NaN,NaN,NaN,ACRE
1,2012,8,AL,0.0,0.0,0.0,0.0,417404,110517,0.0,2012-08-01,NaN,NaN,NaN,NaN,ALAGOAS
2,2012,8,AM,0.0,0.0,0.0,0.0,317456,83948,0.0,2012-08-01,NaN,NaN,NaN,NaN,AMAZONAS
3,2012,8,AP,0.0,0.0,0.0,0.0,55886,16010,0.0,2012-08-01,NaN,NaN,NaN,NaN,AMAPÁ
4,2012,8,BA,0.0,0.0,0.0,0.0,1831657,429469,0.0,2012-08-01,NaN,NaN,NaN,NaN,BAHIA


In [11]:
to_date(br)
to_date(uf)
to_date(df)

In [24]:
max_year = datetime.strptime('2022-05-01', '%Y-%m-%d').year
max_month = datetime.strptime('2022-05-01', '%Y-%m-%d').month

In [25]:
uf_cecad.columns = ['UF', 'Estado', 'nao_recebe_pessoas', 'recebe_pessoas', 'sem_resposta', 'total_pessoas']
uf_cecad['ano'] = max_year
uf_cecad['mes'] = max_month
uf_cecad.drop(columns=['Estado'], inplace=True)

In [26]:
uf = uf.merge(uf_cecad, on=['UF', 'ano', 'mes'], how='left').merge(siglas, left_on='UF', right_on='Sigla', how='left')
uf.drop(columns=['Sigla'], inplace=True)

In [27]:
br_cecad.columns = ['Brasil', 'nao_recebe_pessoas', 'recebe_pessoas', 'sem_resposta', 'total_pessoas']
br_cecad['ano'] = max_year
br_cecad['mes'] = max_month

In [31]:
br.head()

,ano,mes,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total,data
0,2012,8,0.0,0.0,0.0,0.0,12897221,5265743,0.0,2012-08-01
1,2012,9,0.0,0.0,0.0,0.0,13006755,5278170,0.0,2012-09-01
2,2012,10,0.0,0.0,0.0,0.0,13076366,5288902,0.0,2012-10-01
3,2012,11,0.0,0.0,0.0,0.0,13131359,5298295,0.0,2012-11-01
4,2012,12,0.0,0.0,0.0,0.0,13161099,5311337,0.0,2012-12-01


In [30]:
br_cecad

,Brasil,nao_recebe_pessoas,recebe_pessoas,sem_resposta,total_pessoas,ano,mes
0,Brasil,7.390.728,48.587.348,0,55.978.076,2022,5


In [ ]:
br = br.merge(br_cecad, on=['ano', 'mes'], how='left')
uf.drop(columns=['Sigla'], inplace=True)

In [43]:
for col in [col for col in uf.columns if 'pessoas' in col]:
    uf[col] = uf[col].apply(lambda x: float(str(x).replace('.', '')))

In [44]:
dt = datetime.datetime.now()
br['ts'] = datetime.date(dt.year, dt.month, dt.day)

In [48]:
br.to_parquet('Results/fam_br.parquet')
uf.to_parquet('Results/fam_uf.parquet')
df.to_parquet('Results/fam_mun.parquet')